## Create 3d models and snapshots with BIDS

In [ ]:
import TPTBox.core.bids_files as bids
from TPTBox.snapshot3D import snapshot_subreg_3d
from TPTBox.core import bids_utils

In [ ]:
# making the bids dataset
in_path = "D:\Repositories\Work/transitional_vertebra\data\data_tv/fxclass"
bids_ds = bids.BIDS_Global_info(
    datasets=[in_path], parents=["rawdata", "derivatives", "cutout"], additional_key=["snp", "ovl"], verbose=False
)

In [ ]:
# iterate over subjects
for name, sample in bids_ds.enumerate_subjects(sort=True):
    # enforce a list of families
    if isinstance(sample, bids.BIDS_Family):
        families = [sample]
    else:
        families = bids_utils.filter_ct_2_family(name, sample, False)
    if families is None:
        continue
    for family in families:
        # get the bids_file for the segmentations
        vert_ref = family.get("msk_seg-vert")
        subreg_ref = family.get("msk_seg-subreg")

        # if at least one of them is not available, skip the sample
        if vert_ref is None or subreg_ref is None:
            print(f"Did not find vert_msk, and/or subreg_msk, in sample {name}, {family.get_key_len()}")
            continue
        # check centroid jsons
        ctd_vert = family.get(["ctd_vert", "ctd_subreg"], None)
        if ctd_vert is None:
            print(f"skipped due to missing json centroid, only got {family.get_key_len()}")
            continue

        # call the make subregion_3d_and_snapshot function
        # it automatically saves the individual 3d models of the vertebra as well as the combined snapshot
        snapshot_subreg_3d.make_subregion_3d_and_snapshot(
            vert_ref[0],  # the vertebra_mask_bids_file
            sub_ref=subreg_ref[0],  # the bids_file for the subregion segmentation
            vert_idx_list=[18, 19, 28, 20, 21],  # which vertebra indices (e.g. 18 = T11) it should take into account
            verbose=False,
            save_combined_model=True,  # whether a combined model of the vertebra is saved
            save_individual_snapshots=True,  # wheter snapshots of the individual subregion segmentations should be saved
        )
    break
    print("Finished")

In [ ]:
# Visualize the snaps in code via Image package
input_snap = "D:/Repositories/Work/transitional_vertebra/data/data_tv/fxclass/derivatives/ctfu00006/ses-20120920/sub-ctfu00006_ses-20120920_seg-subreg_vertebra-T11-L2_snapshot.png"
from IPython.display import Image

pil_img = Image(filename=input_snap)
display(pil_img)